<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Project 2: Kaggle Submission

> Authors: Lim Zheng Gang, Eugene Matthew Cheong, Pius Yee

### Notebook 4. Kaggle Submission
##### In this Notebook 4, we will apply the same data cleaning and feature engineering techniques used on the training data to the test dataset. This will ensure consistency and allow us to generate model results suitable for Kaggle submission.

##### We will proceed to Kaggle submission once generated the result.

### Repeating the same data cleaning and feature engineering on the test dataset as below:-

In [1]:
# import libraries

from datetime import datetime
import math
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import scipy
from sklearn.linear_model import LinearRegression
import category_encoders as ce
from sklearn.preprocessing import StandardScaler
import pickle

from sklearn import metrics

pd.set_option('display.max_columns', None)

In [2]:
# read csv file
hdb_raw = pd.read_csv("../datasets/test.csv", low_memory=False)

In [3]:
def calculate_distance(lat1, lng1, lat2, lng2):
    # Your distance calculation function (you can use haversine or any other)
    # Example using haversine formula:
    R = 6371  # Earth radius in kilometers
    dlat = np.radians(lat2 - lat1)
    dlng = np.radians(lng2 - lng1)
    a = np.sin(dlat / 2) * np.sin(dlat / 2) + np.cos(np.radians(lat1)) * np.cos(np.radians(lat2)) * np.sin(dlng / 2) * np.sin(dlng / 2)
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    distance_km = R * c
    return distance_km

Clean up data and remove null values

In [4]:
# to replace said columns with 0

hdb_raw["Mall_Within_500m"] = hdb_raw["Mall_Within_500m"].map(lambda x : 0 if np.isnan(x) else x)
hdb_raw["Mall_Within_1km"] = hdb_raw["Mall_Within_1km"].map(lambda x : 0 if np.isnan(x) else x)
hdb_raw["Mall_Within_2km"] = hdb_raw["Mall_Within_2km"].map(lambda x : 0 if np.isnan(x) else x) 
hdb_raw["Hawker_Within_500m"] = hdb_raw["Hawker_Within_500m"].map(lambda x : 0 if np.isnan(x) else x)
hdb_raw["Hawker_Within_1km"] = hdb_raw["Hawker_Within_1km"].map(lambda x : 0 if np.isnan(x) else x)
hdb_raw["Hawker_Within_2km"] = hdb_raw["Hawker_Within_2km"].map(lambda x : 0 if np.isnan(x) else x)

In [5]:

col_Y_N = ['commercial','market_hawker','multistorey_carpark','precinct_pavilion']

for col in col_Y_N:
    hdb_raw[col] = hdb_raw[col].str.replace('N','0')
    hdb_raw[col] = hdb_raw[col].str.replace('Y','1').astype("int")

In [6]:
hdb_raw["tenure"] = hdb_raw["Tranc_Year"] - hdb_raw["lease_commence_date"]
bin_edges = list(range(0, hdb_raw["tenure"].max() + 6, 5))
bin_edges = [0,11, hdb_raw["tenure"].max() + 1]
bin_labels = [f"{bin_edges[i]}-{bin_edges[i+1]-1}" for i in range(len(bin_edges)-1)]
hdb_raw["tenure_buckets"] = pd.cut(hdb_raw["tenure"], bins=bin_edges, labels=bin_labels, include_lowest=True, right=False)
hdb_raw = pd.get_dummies(hdb_raw, columns=['tenure_buckets'], prefix='tenure_buckets')

In [7]:
# calculate distance from centre - choose 79 Anson Road (1.2742738490494008, 103.84567324086873)

lat1 = 1.2742738490494008
lng1 =  103.84567324086873
hdb_raw["from_centre_distance"] = calculate_distance(lat1, lng1, hdb_raw["Latitude"], hdb_raw["Longitude"])

In [8]:
hdb_raw["is_pre_war"] = hdb_raw["year_completed"].map(lambda x : 1 if x == 1949 else 0)

In [9]:

df_sm_location = pd.read_csv("../datasets/sm_location_supermalls.csv")

df_mh_calculation = pd.DataFrame()
df_mh_calculation["id"] = hdb_raw["id"]
df_mh_calculation["lat"] = hdb_raw["Latitude"]
df_mh_calculation["lng"] = hdb_raw["Longitude"]

for each_id in df_mh_calculation["id"].values:
    lat1 = df_mh_calculation.loc[df_mh_calculation[df_mh_calculation["id"]==each_id].index, "lat"].iloc[0]
    lng1 = df_mh_calculation.loc[df_mh_calculation[df_mh_calculation["id"]==each_id].index, "lng"].iloc[0]

    latitudes = df_sm_location["LATITUDE"].values
    longitudes = df_sm_location["LONGITUDE"].values

    distances = calculate_distance(lat1, lng1, latitudes, longitudes)

    min_dist_mall = np.min(distances)

    df_mh_calculation.loc[df_mh_calculation[df_mh_calculation["id"]==each_id].index, "mh"] = min_dist_mall
    
df_mh_calculation.to_csv('../datasets/supermh_test.csv', index=False) # to avoid running above loop again
hdb_raw = pd.merge(hdb_raw, df_mh_calculation[['id', 'mh']], on='id', how='left')

In [10]:
hdb_raw["is_premium"] = hdb_raw["flat_model"].map(lambda x: 1 if x in  ("Improved-Maisonette","DBSS") else 0)
hdb_raw["is_terrace"] = hdb_raw["flat_model"].map(lambda x: 1 if x ==  "Terrace" else 0)


In [11]:
hdb_raw["is_superlargeterrace"] = hdb_raw["floor_area_sqft"].map(lambda x: 1 if x >= 2250 else 0)

In [12]:
# new column for mature/non mature estate
mature_lst = ["Ang Mo Kio","Bedok","Bishan","Bukit Merah","Bukit Timah","Clementi","Downtown Core","Geylang","Kallang","Marine Parade", "Novena", "Outram", "Pasir Ris", "Queenstown", "Rochor","Serangoon","Tampines","Tanglin","Toa Payoh"]
area_category_mapping = {
    'Tanglin': "Group1",
    'Bukit Timah': "Group1",
    'Outram': "Group1",
    'Downtown Core': "Group1",
    'Bishan': "Group1",
    'Bukit Merah': "GroupJB",
    'Queenstown': "GroupCQS",
    'Marine Parade': "GroupCM",
    'Serangoon': "GroupCQS",
    'Clementi': "GroupCQS",
    'Hougang': "GroupYH",
    'Bukit Panjang': "GroupPWC",
    'Western Water Catchment': "Group2",
    'Jurong East': "GroupJB",
    'Ang Mo Kio': "GroupA",
    'Choa Chu Kang': "GroupPWC",
    'Woodlands': "GroupPWC",
    'Yishun': "GroupYH",
    'Changi': "GroupCM"
}

hdb_raw["mature"] = hdb_raw["planning_area"].apply(lambda x: 1 if x in mature_lst else 0)
hdb_raw['planning_area_category'] = hdb_raw['planning_area'].map(area_category_mapping)
hdb_raw = pd.get_dummies(hdb_raw, columns=['planning_area_category'], prefix='planning_area_category')


In [13]:
year_category_mapping = {
    2014: "Group2",
    2015: "Group1",
    2016: "Group1",
    2017: "Group2",
    2018: "Group1",
    2019: "Group0",
    2020: "Group2",

}

hdb_raw['year_category'] = hdb_raw['Tranc_Year'].map(year_category_mapping)
hdb_raw = pd.get_dummies(hdb_raw, columns=['year_category'], prefix='year_category')

In [14]:
# Identify boolean columns
bool_columns = hdb_raw.select_dtypes(include='bool').columns

# Convert boolean columns to int
hdb_raw[bool_columns] = hdb_raw[bool_columns].astype(int)

In [15]:
hdb_raw = pd.get_dummies(hdb_raw, columns=['flat_type'], prefix='flat_type')

In [16]:
columns_to_drop = [
    "Tranc_YearMonth", "town", "block", "price_per_sqft", "street_name", "storey_range", "floor_area_sqm", 
    "mid_storey", "lower", "full_flat_type", "address", "residential", "1room_rental", "2room_rental", 
    "3room_rental", "other_room_rental", "postal", "mrt_name", "mrt_latitude", "mrt_longitude", 
    "bus_stop_name", "bus_stop_latitude", "bus_stop_longitude", "pri_sch_name", "pri_sch_latitude", 
    "pri_sch_longitude", "sec_sch_name", "sec_sch_latitude", "sec_sch_longitude", 'multistorey_carpark', 
    'precinct_pavilion', 'hawker_food_stalls', 'hawker_market_stalls', 'bus_interchange', 'cutoff_point', 
    'vacancy', 'flat_model', 'planning_area', 'lease_commence_date', 'Tranc_Year', 'Tranc_Month', 'upper', 'hdb_age', 'year_completed', 'commercial', 'market_hawker', 
    'total_dwelling_units', '1room_sold', '2room_sold', '3room_sold', '4room_sold', '5room_sold', 
    'exec_sold', 'multigen_sold', 'studio_apartment_sold', 'Latitude', 'Longitude', 'Mall_Nearest_Distance', 
    'Mall_Within_500m', 'Mall_Within_1km', 'Mall_Within_2km', 'Hawker_Nearest_Distance', 'Hawker_Within_500m', 
    'Hawker_Within_1km', 'Hawker_Within_2km', 'bus_stop_nearest_distance', 'pri_sch_nearest_distance', 
    'pri_sch_affiliation', 'sec_sch_nearest_dist', 'affiliation', 'tenure_buckets_11-54', 'flat_type_4 ROOM', 'flat_type_5 ROOM', 
    'flat_type_EXECUTIVE', 'flat_type_MULTI-GENERATION','flat_type_3 ROOM', 'mrt_interchange', 'planning_area_category_Group2'
]


hdb_cleaned = hdb_raw.drop(columns = columns_to_drop)


In [17]:
hdb_cleaned.columns

Index(['id', 'mid', 'floor_area_sqft', 'max_floor_lvl', 'mrt_nearest_distance',
       'tenure', 'tenure_buckets_0-10', 'from_centre_distance', 'is_pre_war',
       'mh', 'is_premium', 'is_terrace', 'is_superlargeterrace', 'mature',
       'planning_area_category_Group1', 'planning_area_category_GroupA',
       'planning_area_category_GroupCM', 'planning_area_category_GroupCQS',
       'planning_area_category_GroupJB', 'planning_area_category_GroupPWC',
       'planning_area_category_GroupYH', 'year_category_Group0',
       'year_category_Group1', 'year_category_Group2', 'flat_type_1 ROOM',
       'flat_type_2 ROOM'],
      dtype='object')

In [18]:
X = hdb_cleaned.loc[:, [ 'mid', 'floor_area_sqft', 'max_floor_lvl',
       'mrt_nearest_distance', 'mature', 'planning_area_category_Group1',
       'planning_area_category_GroupA', 'planning_area_category_GroupCM',
       'planning_area_category_GroupCQS', 'planning_area_category_GroupJB',
       'planning_area_category_GroupPWC', 'planning_area_category_GroupYH',
       'tenure', 'tenure_buckets_0-10', 'year_category_Group0',
       'year_category_Group1', 'year_category_Group2', 'is_premium',
       'is_terrace', 'is_superlargeterrace', 'flat_type_1 ROOM',
       'flat_type_2 ROOM', 'is_pre_war', 'from_centre_distance', 'mh']]

### import pickle

In [19]:
with open('../datasets/lasso.pkl', 'rb') as handle:
    lasso_cv = pickle.load(handle)

In [20]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)
y_pred = lasso_cv.predict(X_scaled_df)

In [21]:
hdb_cleaned["id"].shape

(16737,)

In [22]:
y_pred_adjust = np.exp(y_pred)-1

In [23]:
type(y_pred_adjust)

numpy.ndarray

In [24]:
submission = pd.DataFrame()

submission["Id"] = hdb_cleaned["id"]
submission["Predicted"] = y_pred_adjust


### export the Kaggle submission CSV

In [25]:
submission.to_csv('../datasets/submission_kaggle.csv', index=False)

### Kaggle submission done

<img src="../img/submission.png">